In [6]:
import torch, torchvision

from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim
import sys

print("Torch version:", torch.__version__)
print("Torchvision version:", torchvision.__version__)
print("python version:", sys.version)

Torch version: 2.5.1+cu124
Torchvision version: 0.20.1+cu124
python version: 3.10.15 | packaged by Anaconda, Inc. | (main, Oct  3 2024, 07:22:19) [MSC v.1929 64 bit (AMD64)]


In [ ]:
# 准备数据集
batch_size = 64
transform = transforms.Compose([
    transforms.ToTensor(), 
    transforms.Normalize((0.1307,), (0.3081,))
])  # 归一化,均值和方差

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)

# 设计模型使用类
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.layers = torch.nn.Sequential(
            torch.nn.Linear(784, 512),
            torch.nn.ReLU(),
            torch.nn.Linear(512, 256),
            torch.nn.ReLU(),
            torch.nn.Linear(256, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, 10)
        )

    def forward(self, x):
        x = x.view(-1, 784)  # -1其实就是自动获取mini_batch
        return self.layers(x)  

model = Net()

# 构造损失函数和优化器
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)
#optimizer = optim.Adam(model.parameters(), lr=0.01)

# 训练循环前向，反向，更新
def train(epoch):
    running_loss = 0.0
    for batch_idx, (inputs, target) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if batch_idx % 300 == 299:
            print(f'[{epoch + 1}, {batch_idx + 1:5}] loss: {running_loss / 300:.3f}')
            running_loss = 0.0

def test():
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, dim=1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = 100 * correct / total if total > 0 else 0
    print(f'accuracy on test set: {accuracy:.2f} %')

if __name__ == '__main__':
    try:
        for epoch in range(10):
            train(epoch)
            test()
    except Exception as e:
        print(f'发生错误: {e}')


[1,   300] loss: 2.212
[1,   600] loss: 0.905
[1,   900] loss: 0.423
accuracy on test set: 89.58 %
[2,   300] loss: 0.311
[2,   600] loss: 0.262
[2,   900] loss: 0.221
accuracy on test set: 94.22 %
[3,   300] loss: 0.180
[3,   600] loss: 0.169
[3,   900] loss: 0.147
accuracy on test set: 95.74 %
[4,   300] loss: 0.121
[4,   600] loss: 0.127
[4,   900] loss: 0.117
accuracy on test set: 96.38 %
[5,   300] loss: 0.096
[5,   600] loss: 0.094
[5,   900] loss: 0.090
accuracy on test set: 96.91 %
[6,   300] loss: 0.074
[6,   600] loss: 0.070
[6,   900] loss: 0.078
accuracy on test set: 96.78 %
[7,   300] loss: 0.061
[7,   600] loss: 0.061
[7,   900] loss: 0.060
accuracy on test set: 97.49 %
[8,   300] loss: 0.047
[8,   600] loss: 0.049
[8,   900] loss: 0.050
accuracy on test set: 97.65 %
[9,   300] loss: 0.040
[9,   600] loss: 0.039
[9,   900] loss: 0.040
accuracy on test set: 97.70 %
[10,   300] loss: 0.032
[10,   600] loss: 0.033
[10,   900] loss: 0.031
accuracy on test set: 97.10 %
